In [1]:
import numpy as np
import open3d as o3d
import os 
from tqdm.notebook import tqdm
import torch

In [2]:
dirt = 'shape_net_core_uniform_samples_2048/'
folders = os.listdir(dirt)

In [3]:
files = []
for fold in folders:
    direc = dirt + fold
    for file in os.listdir(direc):
        files.append(direc + '/' + file)

In [4]:
len(files)

57449

In [88]:
files[0]

'shape_net_core_uniform_samples_2048/02691156/10155655850468db78d106ce0a280f87.ply'

In [87]:
# Read .ply file
input_file = files[10]
pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud

# Visualize the point cloud within open3d
o3d.visualization.draw_geometries([pcd]) 

# Convert open3d format to numpy array
# Here, you have the point cloud in numpy format. 
point_cloud_in_numpy = np.asarray(pcd.points) 

In [34]:
point_cloud_in_numpy

array([[-0.10635921,  0.06259143, -0.11650325],
       [-0.10808733,  0.05918296, -0.11650223],
       [-0.10937055,  0.07403795, -0.11650145],
       ...,
       [-0.44566393, -0.17004111,  0.11348417],
       [-0.4493975 , -0.17151845,  0.11460905],
       [-0.45363909,  0.17140326,  0.11650325]])

In [5]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self):
        super().__init__()
        pts = []
        for input_file in tqdm(files):
            pcd = o3d.io.read_point_cloud(input_file)
            point_cloud_in_numpy = np.asarray(pcd.points) 
            pts.append(point_cloud_in_numpy)
            
        #maxlen = max([len(x) for x in pts])
        #pad_pts = np.zeros((len(pts), maxlen, 3))
            
        self.pts = pts
        
    def __len__(self):
        return len(self.pts)
    def __getitem__(self, idx):
        pts = self.pts[idx]
        pts = torch.tensor(pts.T).float().contiguous()
        return pts

In [6]:
%load_ext autoreload
%autoreload 2
from PointNetGAN import *

In [7]:
dataset = TrainDataset()
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True, drop_last=True)

  0%|          | 0/57449 [00:00<?, ?it/s]

In [10]:
inputs = next(iter(train_loader))

In [11]:
inputs.shape

torch.Size([32, 3, 2048])

In [12]:
def evaluate_chamfer(xyz1, xyz2):
    xyz1 = xyz1.permute(0, 2, 1)
    xyz2 = xyz2.permute(0, 2, 1)
    distance = torch.cdist(xyz1, xyz2)
    dist1, idx1 = distance.min(2)
    dist2, idx2 = distance.min(1)
    loss = (dist1 ** 2).mean() + (dist2 ** 2).mean()
    return loss

In [ ]:
dist1, idx1, dist2, idx2 = chamfer_distance(src.unsqueeze(0), tgt.unsqueeze(0))
# loss = dist1.mean() + dist2.mean()
loss = (dist1 ** 2).mean() + (dist2 ** 2).mean()

In [37]:
evaluate

[0.034281112253665924,
 0.040341414511203766,
 0.0397404208779335,
 0.04547237232327461,
 0.046153292059898376]

In [35]:
loss_metric[19]

0.12878865141727674

In [38]:
loss_metric

[1.562645170889525,
 0.900783295252861,
 0.7179492243817258,
 0.6486176861693932,
 0.6220264026381511,
 0.5778169152962465,
 0.527606252093169,
 0.4604400806855358,
 0.4179270237186161,
 0.3877804005378468,
 0.3956445123076771,
 0.3519543801592585,
 0.3246562622233312,
 0.2685956906055134,
 0.16303758530340529,
 0.22369547342767415,
 -0.06466052273267615,
 -0.5278507889453373,
 -0.7749239769543737,
 0.44411890549081945]

In [16]:
generator = gen
Tensor = torch.cuda.FloatTensor
generator.eval()
with torch.no_grad():
    for i, (imgs) in enumerate(train_loader):
        real_imgs = Variable(imgs.type(Tensor))
        z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
        fake_imgs = generator(z)
        break
    metric = evaluate_chamfer(real_imgs, fake_imgs)
    generator.train()

In [29]:
xyz1 = real_imgs
xyz2 =fake_imgs
distance = torch.cdist(xyz1, xyz2)
dist1, idx1 = distance.min(2)
dist2, idx2 = distance.min(1)
loss = (dist1 ** 2).mean() + (dist2 ** 2).mean()

In [30]:
loss

tensor(212.6685, device='cuda:0', grad_fn=<AddBackward0>)

In [19]:
metric.data

tensor(0.0347, device='cuda:0')

In [20]:
metric.item()

0.04104997590184212

In [13]:
loss_metric

[-0.49000755100407534,
 -0.009468605200850612,
 -0.02194154486318843,
 -0.048104785403443706,
 -0.04740284590257539,
 -0.03603882079737054,
 -0.040245410880177385,
 -0.043948019673633903,
 0.0008668985683470964,
 -0.05628052438971483,
 -0.7757951567570368,
 -1.6108279943466186,
 -3.017255633407169,
 -3.6930554681354097,
 -3.7768773555755617,
 -6.686784185303582,
 -12.235858350329929,
 -15.29523401260376,
 -15.274354320102267,
 -10.244890133539835,
 -5.836242448786894,
 -2.4515038874414232,
 0.2048629093501303,
 6.968668101231257,
 15.522910300890604,
 16.41706007056766,
 16.052669774161444,
 15.656219658586714,
 13.768474777539572,
 9.874130595723788,
 2.1358176979753707,
 0.8243151863416036,
 -1.0084337830543517,
 -2.157898447248671,
 -0.4445760687192281,
 4.463985275559955,
 6.225847208499909,
 16.68866632911894,
 22.19987750980589,
 29.300741457939147,
 31.208191294140285,
 30.182671615812513,
 33.536974843343096,
 38.190496905644736,
 50.38305331336127,
 43.08859338760376,
 41.4189

In [41]:
z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
generated_pcd = generator(z)

In [54]:
pcd = generated_pcd[10]

In [55]:
pcd = generated_pcd[10]
points = pcd.permute(1,0).cpu().detach().numpy()

In [56]:
points.shape

(2048, 3)

In [57]:
pcd  = o3d.geometry.PointCloud()
points = o3d.utility.Vector3dVector(points) 
pcd.points = points
# Visualize the point cloud within open3d
o3d.visualization.draw_geometries([pcd]) 

In [13]:
lambda_gp = 10
n_critic = 5
generator = Generator()
discriminator = Discriminator()
num_batch = len(dataset)/32

generator.cuda()
discriminator.cuda()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=(0, 0.9))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=2e-4, betas=(0, 0.9))
Tensor = torch.cuda.FloatTensor 
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lambda epoch: 0.98**epoch)
lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_D, lambda epoch: 0.98**epoch)

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    #print(D)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    #print('grad_out', fake.shape)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.contiguous().view(gradients.size(0), -1) #.contiguous()
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

loss_metric = []
evaluate = []
for epoch in tqdm(range(20)):
    loss_epoch = []
    for i, (imgs) in enumerate(train_loader):

        real_imgs = Variable(imgs.type(Tensor))
        optimizer_D.zero_grad()
        z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
        fake_imgs = generator(z)
        real_validity = discriminator(real_imgs)
        fake_validity = discriminator(fake_imgs)
       # print()
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()
        optimizer_G.zero_grad()


        if i % n_critic == 0:

            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()
            loss_epoch.append(g_loss.item())
            print('[%d: %d/%d] train lossD: %f lossG: %f' %(epoch, i, num_batch, d_loss.item(), g_loss.item()))
    
    loss_metric.append(np.mean(loss_epoch))
    lr_scheduler_G.step()
    lr_scheduler_D.step()
  

    generator.eval()
    with torch.no_grad():
        for i, (imgs) in enumerate(train_loader):
            real_imgs = Variable(imgs.type(Tensor))
            z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
            fake_imgs = generator(z)
            metric = evaluate_chamfer(real_imgs, fake_imgs)
            print(metric.item())
            break
        evaluate.append(metric.item())
        generator.train()

  0%|          | 0/20 [00:00<?, ?it/s]

[0: 0/1795] train lossD: 3.371928 lossG: 0.491549
[0: 5/1795] train lossD: -0.340641 lossG: 0.484702
[0: 10/1795] train lossD: -0.170919 lossG: 1.148336
[0: 15/1795] train lossD: -0.220001 lossG: 1.490112
[0: 20/1795] train lossD: -0.384851 lossG: 0.939465
[0: 25/1795] train lossD: -0.458481 lossG: 1.281631
[0: 30/1795] train lossD: -0.529572 lossG: 1.424161
[0: 35/1795] train lossD: -0.533533 lossG: 1.509425
[0: 40/1795] train lossD: 0.128809 lossG: 0.169563
[0: 45/1795] train lossD: 0.021605 lossG: 1.456745
[0: 50/1795] train lossD: -0.638828 lossG: 2.023964
[0: 55/1795] train lossD: -0.794438 lossG: 2.552546
[0: 60/1795] train lossD: -0.840939 lossG: 1.806798
[0: 65/1795] train lossD: -0.870644 lossG: 2.019814
[0: 70/1795] train lossD: -0.257118 lossG: 0.519526
[0: 75/1795] train lossD: -0.070141 lossG: 0.905566
[0: 80/1795] train lossD: -0.524901 lossG: 1.840304
[0: 85/1795] train lossD: 0.334472 lossG: 0.657134
[0: 90/1795] train lossD: -0.708794 lossG: 1.968932
[0: 95/1795] train

[0: 780/1795] train lossD: -0.430038 lossG: 1.325595
[0: 785/1795] train lossD: -0.743533 lossG: 1.246795
[0: 790/1795] train lossD: 0.229032 lossG: 0.872548
[0: 795/1795] train lossD: -0.686428 lossG: 0.929610
[0: 800/1795] train lossD: 0.001197 lossG: 0.628793
[0: 805/1795] train lossD: -0.526242 lossG: 1.311770
[0: 810/1795] train lossD: -0.612677 lossG: 1.296131
[0: 815/1795] train lossD: -0.408852 lossG: 1.379410
[0: 820/1795] train lossD: -0.552614 lossG: 1.379232
[0: 825/1795] train lossD: -0.297291 lossG: 1.501062
[0: 830/1795] train lossD: -0.602518 lossG: 0.882540
[0: 835/1795] train lossD: -0.642057 lossG: 0.868655
[0: 840/1795] train lossD: -0.279704 lossG: 0.757612
[0: 845/1795] train lossD: -0.566835 lossG: 1.010291
[0: 850/1795] train lossD: -0.281643 lossG: 1.161015
[0: 855/1795] train lossD: -0.372771 lossG: 1.092130
[0: 860/1795] train lossD: -0.451295 lossG: 1.403988
[0: 865/1795] train lossD: -0.124796 lossG: 0.931784
[0: 870/1795] train lossD: -0.522037 lossG: 1.07

[0: 1545/1795] train lossD: -0.354695 lossG: 1.172195
[0: 1550/1795] train lossD: -0.084464 lossG: 1.152655
[0: 1555/1795] train lossD: -0.507558 lossG: 1.297679
[0: 1560/1795] train lossD: -0.122080 lossG: 0.884212
[0: 1565/1795] train lossD: -0.423525 lossG: 1.215957
[0: 1570/1795] train lossD: -0.355561 lossG: 1.356374
[0: 1575/1795] train lossD: -0.343442 lossG: 1.515525
[0: 1580/1795] train lossD: -0.543878 lossG: 1.276631
[0: 1585/1795] train lossD: -0.488059 lossG: 1.340485
[0: 1590/1795] train lossD: -0.501343 lossG: 1.550645
[0: 1595/1795] train lossD: -0.354919 lossG: 1.060756
[0: 1600/1795] train lossD: -0.358170 lossG: 1.238329
[0: 1605/1795] train lossD: -0.480648 lossG: 0.988510
[0: 1610/1795] train lossD: -0.487187 lossG: 1.213575
[0: 1615/1795] train lossD: -0.433031 lossG: 0.906981
[0: 1620/1795] train lossD: -0.324433 lossG: 0.716878
[0: 1625/1795] train lossD: -0.299406 lossG: 0.825016
[0: 1630/1795] train lossD: 0.103159 lossG: 0.649324
[0: 1635/1795] train lossD: -

[1: 520/1795] train lossD: -0.371787 lossG: 0.810476
[1: 525/1795] train lossD: -0.464386 lossG: 0.930957
[1: 530/1795] train lossD: -0.258727 lossG: 1.009196
[1: 535/1795] train lossD: -0.104440 lossG: 0.608269
[1: 540/1795] train lossD: -0.434098 lossG: 0.926489
[1: 545/1795] train lossD: -0.336961 lossG: 0.806462
[1: 550/1795] train lossD: -0.479607 lossG: 1.123375
[1: 555/1795] train lossD: -0.255199 lossG: 0.733951
[1: 560/1795] train lossD: -0.306852 lossG: 0.922632
[1: 565/1795] train lossD: -0.366436 lossG: 1.078295
[1: 570/1795] train lossD: -0.371630 lossG: 1.040999
[1: 575/1795] train lossD: -0.399586 lossG: 0.938664
[1: 580/1795] train lossD: -0.411580 lossG: 1.020504
[1: 585/1795] train lossD: -0.154236 lossG: 0.803470
[1: 590/1795] train lossD: -0.356792 lossG: 0.860609
[1: 595/1795] train lossD: -0.473601 lossG: 1.062231
[1: 600/1795] train lossD: -0.319587 lossG: 0.902887
[1: 605/1795] train lossD: -0.413036 lossG: 0.977973
[1: 610/1795] train lossD: -0.328424 lossG: 1.

...

[18: 490/1795] train lossD: -0.019292 lossG: 0.230492
[18: 495/1795] train lossD: -0.045127 lossG: 0.249728
[18: 500/1795] train lossD: -0.179139 lossG: 0.278860
[18: 505/1795] train lossD: -0.064570 lossG: 0.369058
[18: 510/1795] train lossD: -0.179615 lossG: 0.378618
[18: 515/1795] train lossD: -0.451042 lossG: 0.320763
[18: 520/1795] train lossD: -0.318421 lossG: 0.129147
[18: 525/1795] train lossD: -0.124276 lossG: 0.118754
[18: 530/1795] train lossD: 1.182385 lossG: 0.147849
[18: 535/1795] train lossD: -0.009624 lossG: 0.210843
[18: 540/1795] train lossD: 0.012322 lossG: 0.218553
[18: 545/1795] train lossD: -0.050400 lossG: 0.304565
[18: 550/1795] train lossD: -0.043008 lossG: 0.323625
[18: 555/1795] train lossD: -0.008599 lossG: 0.079105
[18: 560/1795] train lossD: -0.455587 lossG: 0.070634
[18: 565/1795] train lossD: -0.035474 lossG: 0.344808
[18: 570/1795] train lossD: -0.107507 lossG: 0.076485
[18: 575/1795] train lossD: 0.014154 lossG: 0.267030
[18: 580/1795] train lossD: -0.

[18: 1250/1795] train lossD: -0.226373 lossG: 0.252055
[18: 1255/1795] train lossD: -0.504007 lossG: 0.275468
[18: 1260/1795] train lossD: -0.176210 lossG: 0.265269
[18: 1265/1795] train lossD: -0.214342 lossG: 0.030674
[18: 1270/1795] train lossD: -0.081049 lossG: 0.306308
[18: 1275/1795] train lossD: -0.716286 lossG: 0.555692
[18: 1280/1795] train lossD: -0.157814 lossG: 0.220224
[18: 1285/1795] train lossD: -0.213651 lossG: 0.354557
[18: 1290/1795] train lossD: -0.024037 lossG: 0.300944
[18: 1295/1795] train lossD: -0.254009 lossG: 0.164366
[18: 1300/1795] train lossD: -0.297053 lossG: 0.054292
[18: 1305/1795] train lossD: -0.252699 lossG: 0.012360
[18: 1310/1795] train lossD: 0.016942 lossG: 0.266159
[18: 1315/1795] train lossD: -0.512041 lossG: 0.311844
[18: 1320/1795] train lossD: -0.300134 lossG: 0.062590
[18: 1325/1795] train lossD: -0.101879 lossG: 0.039655
[18: 1330/1795] train lossD: 0.031284 lossG: 0.115179
[18: 1335/1795] train lossD: -0.009826 lossG: 0.108769
[18: 1340/17

[19: 210/1795] train lossD: 0.075079 lossG: 0.150382
[19: 215/1795] train lossD: 0.012937 lossG: 0.160229
[19: 220/1795] train lossD: -0.029031 lossG: 0.171518
[19: 225/1795] train lossD: 0.016086 lossG: 0.171617
[19: 230/1795] train lossD: 0.039522 lossG: 0.283885
[19: 235/1795] train lossD: -0.148192 lossG: 0.325738
[19: 240/1795] train lossD: -0.058076 lossG: 0.336484
[19: 245/1795] train lossD: -0.200399 lossG: 0.330786
[19: 250/1795] train lossD: -0.005916 lossG: 0.300910
[19: 255/1795] train lossD: -0.145583 lossG: 0.332196
[19: 260/1795] train lossD: -0.301015 lossG: 0.237780
[19: 265/1795] train lossD: -0.595175 lossG: 0.511293
[19: 270/1795] train lossD: -0.522157 lossG: 0.020864
[19: 275/1795] train lossD: -0.363248 lossG: -0.079840
[19: 280/1795] train lossD: -0.031556 lossG: -0.006619
[19: 285/1795] train lossD: -0.167612 lossG: 0.261544
[19: 290/1795] train lossD: -0.464223 lossG: 0.023124
[19: 295/1795] train lossD: 0.183814 lossG: 0.268479
[19: 300/1795] train lossD: -0.

[19: 970/1795] train lossD: -0.054069 lossG: 0.468485
[19: 975/1795] train lossD: -0.350327 lossG: 0.488957
[19: 980/1795] train lossD: -0.115645 lossG: 0.099650
[19: 985/1795] train lossD: -0.834662 lossG: 0.349625
[19: 990/1795] train lossD: -0.525920 lossG: 0.069366
[19: 995/1795] train lossD: -0.074321 lossG: 0.072561
[19: 1000/1795] train lossD: 0.328111 lossG: 0.148121
[19: 1005/1795] train lossD: 0.029044 lossG: 0.221511
[19: 1010/1795] train lossD: 0.014032 lossG: 0.251582
[19: 1015/1795] train lossD: 0.026670 lossG: 0.301332
[19: 1020/1795] train lossD: 0.039980 lossG: 0.252594
[19: 1025/1795] train lossD: -0.066065 lossG: 0.296454
[19: 1030/1795] train lossD: -0.047593 lossG: 0.365610
[19: 1035/1795] train lossD: -0.142934 lossG: 0.292907
[19: 1040/1795] train lossD: -0.026234 lossG: 0.244143
[19: 1045/1795] train lossD: -0.077374 lossG: 0.267684
[19: 1050/1795] train lossD: -0.618631 lossG: 0.121913
[19: 1055/1795] train lossD: -0.102498 lossG: 0.140343
[19: 1060/1795] train

[19: 1720/1795] train lossD: -0.132802 lossG: 0.286624
[19: 1725/1795] train lossD: -0.296394 lossG: 0.411649
[19: 1730/1795] train lossD: -0.046704 lossG: 0.486944
[19: 1735/1795] train lossD: -0.324580 lossG: 0.033480
[19: 1740/1795] train lossD: -0.200020 lossG: -0.249516
[19: 1745/1795] train lossD: -0.295603 lossG: 0.263623
[19: 1750/1795] train lossD: -0.828268 lossG: 0.346242
[19: 1755/1795] train lossD: -0.293528 lossG: 0.278897
[19: 1760/1795] train lossD: -0.410679 lossG: 0.446373
[19: 1765/1795] train lossD: -0.065221 lossG: 0.111475
[19: 1770/1795] train lossD: -0.656395 lossG: 0.343412
[19: 1775/1795] train lossD: 0.367184 lossG: 0.431312
[19: 1780/1795] train lossD: -0.016714 lossG: 0.444727
[19: 1785/1795] train lossD: 0.075902 lossG: 0.410235
[19: 1790/1795] train lossD: -0.620729 lossG: 0.068872
0.04144319146871567


In [14]:
evaluate

[0.04706810414791107,
 0.04150030016899109,
 0.050314679741859436,
 0.04303291440010071,
 0.03097650408744812,
 0.037385858595371246,
 0.04351937025785446,
 0.038115449249744415,
 0.03754439949989319,
 0.03710556402802467,
 0.05840221047401428,
 0.04416750371456146,
 0.04391780495643616,
 0.04444083571434021,
 0.035055823624134064,
 0.05547195300459862,
 0.05058055743575096,
 0.044261276721954346,
 0.04403220862150192,
 0.04144319146871567]

In [15]:
loss_metric

[1.4323050262287134,
 0.9222088059343003,
 0.7767139810207494,
 0.7275533637960642,
 0.6970954487582767,
 0.6279972803293828,
 0.506528032440329,
 0.45121871902085947,
 0.45607200335493325,
 0.44971108478091887,
 0.4427383228264811,
 0.4037037217019328,
 0.37972715563428766,
 0.37237212041128315,
 0.3517938849819736,
 0.37264720304845766,
 0.37103379194035835,
 0.3482412675546072,
 0.26491313311984777,
 0.18868337146040248]

In [ ]:
generator.eval()
with torch.no_grad():
    for i, (imgs) in enumerate(train_loader):
        real_imgs = Variable(imgs.type(Tensor))
        z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
        fake_imgs = generator(z)
        break
    metric = evaluate_chamfer(real_imgs, fake_imgs)
    #generator.train()

In [16]:
generator.eval()
with torch.no_grad():
    z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
    generated_pcd = generator(z)

In [21]:
pcd = generated_pcd[16]
points = pcd.permute(1,0).cpu().detach().numpy()

In [22]:
pcd = o3d.geometry.PointCloud()
points = o3d.utility.Vector3dVector(points) 
pcd.points = points
# Visualize the point cloud within open3d
o3d.visualization.draw_geometries([pcd]) 

In [23]:
model_dict = {'D': discriminator.state_dict(), 'D_O': optimizer_D.state_dict(), 'G': generator.state_dict(), 'G_O': optimizer_G.state_dict()}

In [24]:
torch.save(model_dict, 'PGAN_20.pt')

In [25]:
generator.train()

Generator(
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=6144, bias=True)
  (th): Tanh()
)

In [26]:
#another 20 epochs 
for epoch in tqdm(range(20)):
    loss_epoch = []
    for i, (imgs) in enumerate(train_loader):

        real_imgs = Variable(imgs.type(Tensor))
        optimizer_D.zero_grad()
        z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
        fake_imgs = generator(z)
        real_validity = discriminator(real_imgs)
        fake_validity = discriminator(fake_imgs)
       # print()
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()
        optimizer_G.zero_grad()


        if i % n_critic == 0:

            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()
            loss_epoch.append(g_loss.item())
            print('[%d: %d/%d] train lossD: %f lossG: %f' %(20 + epoch, i, num_batch, d_loss.item(), g_loss.item()))
    
    loss_metric.append(np.mean(loss_epoch))
    lr_scheduler_G.step()
    lr_scheduler_D.step()
  
    generator.eval()
    with torch.no_grad():
        for i, (imgs) in enumerate(train_loader):
            real_imgs = Variable(imgs.type(Tensor))
            z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
            fake_imgs = generator(z)
            metric = evaluate_chamfer(real_imgs, fake_imgs)
            print(metric.item())
            break
        evaluate.append(metric.item())
        generator.train()

  0%|          | 0/20 [00:00<?, ?it/s]

[20: 0/1795] train lossD: -0.807587 lossG: 0.481738
[20: 5/1795] train lossD: -0.748141 lossG: 0.297474
[20: 10/1795] train lossD: -0.106421 lossG: 0.207815
[20: 15/1795] train lossD: -0.424177 lossG: -0.409612
[20: 20/1795] train lossD: -0.578078 lossG: -0.003368
[20: 25/1795] train lossD: -0.028709 lossG: -0.018827
[20: 30/1795] train lossD: 0.097778 lossG: -0.069648
[20: 35/1795] train lossD: -0.024229 lossG: -0.041482
[20: 40/1795] train lossD: -0.114464 lossG: 0.117094
[20: 45/1795] train lossD: -0.177651 lossG: -0.154578
[20: 50/1795] train lossD: -0.610722 lossG: 0.155119
[20: 55/1795] train lossD: 0.169039 lossG: 0.147169
[20: 60/1795] train lossD: -0.046874 lossG: -0.091843
[20: 65/1795] train lossD: -0.246674 lossG: 0.185377
[20: 70/1795] train lossD: -0.892875 lossG: 0.282319
[20: 75/1795] train lossD: 0.109280 lossG: 0.042113
[20: 80/1795] train lossD: 0.047861 lossG: 0.057616
[20: 85/1795] train lossD: 0.031247 lossG: 0.027552
[20: 90/1795] train lossD: 0.008372 lossG: 0.0

[20: 760/1795] train lossD: 0.020829 lossG: -0.284100
[20: 765/1795] train lossD: -0.789840 lossG: -0.411486
[20: 770/1795] train lossD: -1.427090 lossG: -0.806479
[20: 775/1795] train lossD: 0.034753 lossG: 0.189283
[20: 780/1795] train lossD: 0.153099 lossG: 0.264034
[20: 785/1795] train lossD: 0.049522 lossG: 0.330557
[20: 790/1795] train lossD: 0.031899 lossG: 0.396909
[20: 795/1795] train lossD: -0.034588 lossG: 0.316733
[20: 800/1795] train lossD: 0.037472 lossG: 0.208986
[20: 805/1795] train lossD: -0.022307 lossG: 0.046562
[20: 810/1795] train lossD: -0.040782 lossG: 0.061139
[20: 815/1795] train lossD: -0.042128 lossG: 0.148480
[20: 820/1795] train lossD: -0.003080 lossG: 0.080112
[20: 825/1795] train lossD: 0.056688 lossG: -0.140723
[20: 830/1795] train lossD: -0.177413 lossG: -0.128567
[20: 835/1795] train lossD: -0.345335 lossG: -0.409033
[20: 840/1795] train lossD: -0.352528 lossG: -0.186890
[20: 845/1795] train lossD: -0.672054 lossG: -0.019124
[20: 850/1795] train lossD:

[20: 1510/1795] train lossD: 0.229047 lossG: 0.067796
[20: 1515/1795] train lossD: 0.056498 lossG: 0.135144
[20: 1520/1795] train lossD: -0.092086 lossG: 0.177526
[20: 1525/1795] train lossD: 0.003877 lossG: 0.209724
[20: 1530/1795] train lossD: 0.037385 lossG: 0.279255
[20: 1535/1795] train lossD: 0.096546 lossG: 0.262552
[20: 1540/1795] train lossD: 0.015448 lossG: 0.236089
[20: 1545/1795] train lossD: -0.007470 lossG: 0.200547
[20: 1550/1795] train lossD: -0.042142 lossG: 0.244105
[20: 1555/1795] train lossD: -0.069728 lossG: 0.286821
[20: 1560/1795] train lossD: -0.061560 lossG: 0.193381
[20: 1565/1795] train lossD: -0.036469 lossG: 0.228478
[20: 1570/1795] train lossD: -0.069730 lossG: 0.123842
[20: 1575/1795] train lossD: -0.305970 lossG: 0.097855
[20: 1580/1795] train lossD: -0.621070 lossG: -0.116850
[20: 1585/1795] train lossD: -0.390693 lossG: -0.334977
[20: 1590/1795] train lossD: -0.790113 lossG: -0.420916
[20: 1595/1795] train lossD: -1.118239 lossG: 0.110824
[20: 1600/179

[21: 470/1795] train lossD: -0.144023 lossG: 0.447918
[21: 475/1795] train lossD: -0.544629 lossG: 0.181151
[21: 480/1795] train lossD: -0.814790 lossG: 0.239695
[21: 485/1795] train lossD: -1.380573 lossG: -0.017210
[21: 490/1795] train lossD: -0.084046 lossG: 0.112156
[21: 495/1795] train lossD: -0.946914 lossG: -0.136392
[21: 500/1795] train lossD: -0.774300 lossG: 0.082123
[21: 505/1795] train lossD: 0.062043 lossG: 0.143084
[21: 510/1795] train lossD: 0.009030 lossG: 0.228548
[21: 515/1795] train lossD: -0.052567 lossG: 0.393810
[21: 520/1795] train lossD: 0.071534 lossG: 0.359392
[21: 525/1795] train lossD: 0.000448 lossG: 0.364192
[21: 530/1795] train lossD: 0.031893 lossG: 0.278142
[21: 535/1795] train lossD: -0.145217 lossG: 0.049643
[21: 540/1795] train lossD: -0.006338 lossG: 0.262318
[21: 545/1795] train lossD: -0.256832 lossG: 0.048698
[21: 550/1795] train lossD: -0.952073 lossG: 0.513741
[21: 555/1795] train lossD: -0.115281 lossG: -0.000642
[21: 560/1795] train lossD: -0

...

[38: 230/1795] train lossD: -0.037890 lossG: -0.089643
[38: 235/1795] train lossD: -0.011309 lossG: 0.007671
[38: 240/1795] train lossD: 0.087120 lossG: 0.013606
[38: 245/1795] train lossD: 0.007396 lossG: 0.043023
[38: 250/1795] train lossD: 0.063871 lossG: 0.064247
[38: 255/1795] train lossD: -0.156423 lossG: 0.240442
[38: 260/1795] train lossD: -1.405535 lossG: -1.106499
[38: 265/1795] train lossD: 0.368732 lossG: 0.044418
[38: 270/1795] train lossD: 0.239095 lossG: -2.075015
[38: 275/1795] train lossD: 0.204358 lossG: -0.504946
[38: 280/1795] train lossD: 0.193471 lossG: -0.596026
[38: 285/1795] train lossD: 0.052891 lossG: -0.505955
[38: 290/1795] train lossD: -1.354221 lossG: -0.690121
[38: 295/1795] train lossD: -1.908196 lossG: -1.265279
[38: 300/1795] train lossD: 0.874999 lossG: 0.082350
[38: 305/1795] train lossD: -1.511365 lossG: -0.229059
[38: 310/1795] train lossD: -0.199047 lossG: -0.156301
[38: 315/1795] train lossD: -0.006885 lossG: -0.065165
[38: 320/1795] train lossD

[38: 985/1795] train lossD: -4.171470 lossG: -0.384867
[38: 990/1795] train lossD: -3.156004 lossG: -1.189877
[38: 995/1795] train lossD: 0.201452 lossG: -0.314785
[38: 1000/1795] train lossD: 0.131517 lossG: -0.289923
[38: 1005/1795] train lossD: 0.271116 lossG: -0.230807
[38: 1010/1795] train lossD: 0.085539 lossG: -0.143189
[38: 1015/1795] train lossD: 0.034521 lossG: -0.122034
[38: 1020/1795] train lossD: 0.011073 lossG: -0.064953
[38: 1025/1795] train lossD: 0.227806 lossG: -0.130983
[38: 1030/1795] train lossD: 0.204385 lossG: -0.114892
[38: 1035/1795] train lossD: 0.097206 lossG: -0.122048
[38: 1040/1795] train lossD: -0.194720 lossG: -0.105571
[38: 1045/1795] train lossD: -0.041736 lossG: -0.158900
[38: 1050/1795] train lossD: -0.179628 lossG: -0.226566
[38: 1055/1795] train lossD: 69.062233 lossG: -0.130676
[38: 1060/1795] train lossD: -0.167190 lossG: -0.112366
[38: 1065/1795] train lossD: -1.243884 lossG: -0.211814
[38: 1070/1795] train lossD: -1.345820 lossG: -0.185051
[38:

[38: 1725/1795] train lossD: -2.806200 lossG: -1.018286
[38: 1730/1795] train lossD: -2.537774 lossG: -0.157397
[38: 1735/1795] train lossD: 1662.068481 lossG: -0.220517
[38: 1740/1795] train lossD: 0.056082 lossG: -0.298367
[38: 1745/1795] train lossD: 0.017895 lossG: -0.267275
[38: 1750/1795] train lossD: 0.177554 lossG: -0.262192
[38: 1755/1795] train lossD: -0.453121 lossG: -0.246056
[38: 1760/1795] train lossD: -0.373889 lossG: -0.249420
[38: 1765/1795] train lossD: 0.295679 lossG: -0.220449
[38: 1770/1795] train lossD: 0.129634 lossG: -0.234887
[38: 1775/1795] train lossD: 0.035619 lossG: -0.236790
[38: 1780/1795] train lossD: 0.012978 lossG: -0.227303
[38: 1785/1795] train lossD: 0.392684 lossG: -0.265077
[38: 1790/1795] train lossD: 0.118506 lossG: -0.220596
0.036171235144138336
[39: 0/1795] train lossD: 0.179193 lossG: -0.213773
[39: 5/1795] train lossD: 0.291939 lossG: -0.306243
[39: 10/1795] train lossD: 0.032552 lossG: -0.222349
[39: 15/1795] train lossD: 0.064712 lossG: -0

[39: 680/1795] train lossD: -4.108419 lossG: -1.003076
[39: 685/1795] train lossD: -6.377517 lossG: 0.028495
[39: 690/1795] train lossD: 0.149702 lossG: -0.446987
[39: 695/1795] train lossD: 0.162025 lossG: -0.282551
[39: 700/1795] train lossD: -0.069486 lossG: -0.285462
[39: 705/1795] train lossD: -0.899304 lossG: -0.246603
[39: 710/1795] train lossD: -3.143528 lossG: -0.708503
[39: 715/1795] train lossD: -2.685865 lossG: -2.223185
[39: 720/1795] train lossD: -4.906788 lossG: -0.116703
[39: 725/1795] train lossD: -0.471645 lossG: -0.332836
[39: 730/1795] train lossD: -3.705525 lossG: -0.297792
[39: 735/1795] train lossD: -1.738905 lossG: -0.969556
[39: 740/1795] train lossD: -5.900486 lossG: -0.542652
[39: 745/1795] train lossD: -5.773779 lossG: -0.708774
[39: 750/1795] train lossD: -2.640245 lossG: -0.196195
[39: 755/1795] train lossD: -3.864635 lossG: -1.077354
[39: 760/1795] train lossD: -4.692628 lossG: -0.097824
[39: 765/1795] train lossD: -2.068612 lossG: -1.341447
[39: 770/1795

[39: 1420/1795] train lossD: -3.964623 lossG: -0.130455
[39: 1425/1795] train lossD: 0.117460 lossG: -0.212944
[39: 1430/1795] train lossD: 0.002792 lossG: -0.161222
[39: 1435/1795] train lossD: -0.084504 lossG: -0.134999
[39: 1440/1795] train lossD: 0.073458 lossG: -0.142915
[39: 1445/1795] train lossD: -0.018950 lossG: -0.142787
[39: 1450/1795] train lossD: -0.261965 lossG: -0.147242
[39: 1455/1795] train lossD: -0.185123 lossG: -0.203382
[39: 1460/1795] train lossD: 0.239345 lossG: -0.348871
[39: 1465/1795] train lossD: 1.229861 lossG: -0.519055
[39: 1470/1795] train lossD: 1.032183 lossG: -0.674683
[39: 1475/1795] train lossD: -0.294048 lossG: -0.590278
[39: 1480/1795] train lossD: 0.224363 lossG: -0.535990
[39: 1485/1795] train lossD: 0.025119 lossG: -0.360621
[39: 1490/1795] train lossD: -0.971395 lossG: -0.078631
[39: 1495/1795] train lossD: -6.523236 lossG: -0.407102
[39: 1500/1795] train lossD: -5.217040 lossG: -0.170955
[39: 1505/1795] train lossD: -3.642585 lossG: -3.086711


In [27]:
loss_metric

[1.4323050262287134,
 0.9222088059343003,
 0.7767139810207494,
 0.7275533637960642,
 0.6970954487582767,
 0.6279972803293828,
 0.506528032440329,
 0.45121871902085947,
 0.45607200335493325,
 0.44971108478091887,
 0.4427383228264811,
 0.4037037217019328,
 0.37972715563428766,
 0.37237212041128315,
 0.3517938849819736,
 0.37264720304845766,
 0.37103379194035835,
 0.3482412675546072,
 0.26491313311984777,
 0.18868337146040248,
 0.028184454581988223,
 -0.07199223144472808,
 0.16016489515874877,
 0.6743973408900927,
 0.6254090029217074,
 0.6307525320969585,
 0.632965053190428,
 0.5895270662055374,
 0.5368644741252272,
 0.4228716310020277,
 0.3332138168197488,
 0.21974149769467813,
 0.23078663912822278,
 0.2236530573384883,
 0.1598041756260196,
 0.13880675474394497,
 0.1210674116566964,
 -0.09312290773522057,
 -0.335221137138068,
 -0.6389943713472747]

In [28]:
evaluate

[0.04706810414791107,
 0.04150030016899109,
 0.050314679741859436,
 0.04303291440010071,
 0.03097650408744812,
 0.037385858595371246,
 0.04351937025785446,
 0.038115449249744415,
 0.03754439949989319,
 0.03710556402802467,
 0.05840221047401428,
 0.04416750371456146,
 0.04391780495643616,
 0.04444083571434021,
 0.035055823624134064,
 0.05547195300459862,
 0.05058055743575096,
 0.044261276721954346,
 0.04403220862150192,
 0.04144319146871567,
 0.038034774363040924,
 0.03929232805967331,
 0.04248970001935959,
 0.04734976589679718,
 0.047086551785469055,
 0.044641152024269104,
 0.05280192196369171,
 0.042118966579437256,
 0.03937861695885658,
 0.039278384298086166,
 0.050072867423295975,
 0.03799403831362724,
 0.05065019801259041,
 0.043432075530290604,
 0.04405993968248367,
 0.033305034041404724,
 0.04320012032985687,
 0.05068539083003998,
 0.036171235144138336,
 0.030209101736545563]

In [50]:
generator.eval()
with torch.no_grad():
    z = Variable(Tensor(np.random.normal(0, 1, (32, 128))))
    generated_pcd = generator(z)

In [84]:
pcd = generated_pcd[31]
points = pcd.permute(1,0).cpu().detach().numpy()

In [85]:
pcd = o3d.geometry.PointCloud()
points = o3d.utility.Vector3dVector(points) 
pcd.points = points
o3d.visualization.draw_geometries([pcd]) 